In [1]:
# -*- coding: utf-8 -*-

#Convert mutliclasses into Single Class

#Resolution: 0.004

#Created on Mon Apr 11 16:42:38 2022

#@author: li.chao.987@s.kyushu-u.ac.jp


from osgeo import gdal
import numpy as np
import os

In [6]:
aimFolder = "F:\\17_Article\\01_Data\\01_LandCover\\LandCover0004WGS\\"
#os.mkdir("F:\\17_Article\\01_Data\\01_LandCover\\LandCover0004WGS")

year=2001
while year < 2021:
    original_raster = gdal.Open("F:\\17_Article\\01_Data\\01_LandCover\\LandCoverMerge\\LandCover_UMD_" + str(year) + "_500m.tif")
    #print(original_raster)

    raster_rprj = gdal.Warp(aimFolder + "LandCover_UMD_"+ str(year) +"_0004_WGS.tif", original_raster, dstSRS = "EPSG:4326", xRes = 0.004, yRes = 0.004, resampleAlg = "mode")
    print(raster_rprj)
    #raster_rprj = None

    original_raster = None

    band = raster_rprj.GetRasterBand(1)

    multiLista = band.ReadAsArray()

    # reclassification
    singleRasterLocation = "F:/17_Article/01_Data/01_LandCover/LandCoverSingleClass/"
    driver = gdal.GetDriverByName("GTiff")
    aim = 0
    while aim < 16:
        print(aim)
        originalToSingleLista = multiLista.copy()
        orignialToSingleListaClip = originalToSingleLista[:,37750:41750].copy()
        #orignialToSingleListaClip = 
        orignialToSingleListaClip[orignialToSingleListaClip == aim] = 100
        orignialToSingleListaClip[orignialToSingleListaClip != 100] = 0
        
        #for j in range(raster_rprj.RasterXSize):
        #    for i in range(raster_rprj.RasterYSize):
        #        if originalToSingleLista[i, j] == aim:
        #            originalToSingleLista[i, j] = 1
        #        else:
        #            originalToSingleLista[i, j] = 0
                    
        # create new file
        name = "year_" + str(year) + "_class_" + str(aim) + ".tif"
        singleRaster = driver.Create(singleRasterLocation + name, 8000, 7500, 1)
        singleRaster.GetRasterBand(1).WriteArray(orignialToSingleListaClip)
        
        proj = raster_rprj.GetProjection()
        georef = (122.0, 0.004, 0.0, 50.0, 0.0, -0.004)
        singleRaster.SetProjection(proj)
        singleRaster.SetGeoTransform(georef)
        singleRaster.FlushCache()
        singleRaster = None
        aim += 1
        
    raster_rprj = None   

    os.remove(aimFolder + "LandCover_UMD_" + str(year) + "_0004_WGS.tif")

    year = year + 1

None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None


Signature: os.mkdir(path, mode=511, *, dir_fd=None)
Docstring:
Create a directory.

If dir_fd is not None, it should be a file descriptor open to a directory,
  and path should be relative; path will then be relative to that directory.
dir_fd may not be implemented on your platform.
  If it is unavailable, using it will raise a NotImplementedError.

The mode argument is ignored on Windows.
Type:      builtin_function_or_method
